In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import lsst.sims.maf.db as db
import lsst.sims.maf.utils as utils
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
from mafContrib import PeriodicStarMetric 

/Users/yoachim/git_repos/sims_maf_contrib/mafContrib/transientAsciiSEDMetric.py:19: UserWarning: sncosmo not installed
  warnings.warn('sncosmo not installed')
/Users/yoachim/git_repos/sims_maf_contrib/mafContrib/LSSObsStrategy/artificialStructureCalculation.py:40: UserWarning: sympy not installed
  warnings.warn('sympy not installed')


In [2]:
outDir ='starfit'
dbFile = 'baseline_nexp2_v1.7_10yrs.db'
opsimdb = db.OpsimDatabase(dbFile)
resultsDb = db.ResultsDb(outDir=outDir)

In [3]:
# Set RA, Dec for a single point in the sky. in radians.

# Both in degrees 
ra = [283.7, 15.0, 100.4, 40.0, 27.7]
dec = [-30.5, -33.7, -50.9, -34.4, -44.4]

# Let's try 5 different distance modulus at each point
nmod = 5
distMod = np.arange(0, nmod)
distMod = distMod/distMod.max() * 12 + 15


ra = np.repeat(ra, nmod)
dec = np.repeat(dec, nmod)
distMod = np.tile(distMod, nmod)

print('Distance Moduli that will be added to the mean magnitudes', distMod)


Distance Moduli that will be added to the mean magnitudes [15. 18. 21. 24. 27. 15. 18. 21. 24. 27. 15. 18. 21. 24. 27. 15. 18. 21.
 24. 27. 15. 18. 21. 24. 27.]


In [4]:

# Note I turned nMonte way down for speed
metric = PeriodicStarMetric(period=0.2, amplitude=0.9, phase=2.,
                            nMonte=100, periodTol=0.05, ampTol=0.05, means=[3.9, 3.5, 3.4, 3.4, 3.4, 3.4],
                            magTol=0.05, nBands=2)

slicer = slicers.UserPointsSlicer(ra,dec,lonCol='fieldRA',latCol='fieldDec')
slicer.slicePoints['distMod'] = distMod
sql = 'night < 1400'
bundle = metricBundles.MetricBundle(metric,slicer,sql)
bg =  metricBundles.MetricBundleGroup({0:bundle}, opsimdb,
                                      outDir=outDir, resultsDb=resultsDb)

In [5]:
bg.runAll()

Querying database SummaryAllProps with constraint night < 1400 for columns ['fiveSigmaDepth', 'fieldRA', 'filter', 'observationStartMJD', 'fieldDec']
Found 791024 visits
Running:  [0]
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [6]:
bundle.metricValues

masked_array(data=[1.0, 1.0, 0.83, 0.06, 0.0, 1.0, 1.0, 0.83, 0.05, 0.0,
                   1.0, 1.0, 0.94, 0.06, 0.0, 1.0, 1.0, 0.8, 0.02, 0.0,
                   1.0, 1.0, 0.89, 0.03, 0.02],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False],
       fill_value=-666.0)

In [7]:
print('RA,  Dec, m-M, Fraction detected')
for i, dummy in enumerate(ra):
    print(ra[i], dec[i], distMod[i], bundle.metricValues[i])

RA,  Dec, m-M, Fraction detected
283.7 -30.5 15.0 1.0
283.7 -30.5 18.0 1.0
283.7 -30.5 21.0 0.83
283.7 -30.5 24.0 0.06
283.7 -30.5 27.0 0.0
15.0 -33.7 15.0 1.0
15.0 -33.7 18.0 1.0
15.0 -33.7 21.0 0.83
15.0 -33.7 24.0 0.05
15.0 -33.7 27.0 0.0
100.4 -50.9 15.0 1.0
100.4 -50.9 18.0 1.0
100.4 -50.9 21.0 0.94
100.4 -50.9 24.0 0.06
100.4 -50.9 27.0 0.0
40.0 -34.4 15.0 1.0
40.0 -34.4 18.0 1.0
40.0 -34.4 21.0 0.8
40.0 -34.4 24.0 0.02
40.0 -34.4 27.0 0.0
27.7 -44.4 15.0 1.0
27.7 -44.4 18.0 1.0
27.7 -44.4 21.0 0.89
27.7 -44.4 24.0 0.03
27.7 -44.4 27.0 0.02
